In [8]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_pinecone import PineconeVectorStore
from langchain_community.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader,PyPDFLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


from langchain_community.llms import CTransformers
import os
import pinecone
from pinecone import Pinecone
from dotenv import load_dotenv

In [3]:
#os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
load_dotenv()
key= os.getenv("PINECONE_API_KEY")

In [5]:
PINECONE_API_KEY = key
PINECONE_API_ENV = 'gcp-starter'

In [17]:
#extract data func
def load_pdf_data(data):
    loader = DirectoryLoader(data,
                    glob='*.pdf',
                    loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [18]:
from fpdf import FPDF
#from PyPDF2 import PdfFileMerger

def create_pdf(input_file):
    # Create a new FPDF object
    pdf = FPDF()

    # Open the text file and read its contents
    with open(input_file, 'r') as f:
        text = f.read()

    # Add a new page to the PDF
    pdf.add_page()

    # Set the font and font size
    pdf.set_font('Arial', size=12)

    # Write the text to the PDF
    pdf.write(5, text)

    # Save the PDF
    pdf.output('gary_timeSt_yt_api.pdf')

#create_pdf("../data/yt_test/gary_timeSt_yt_api.txt")


In [19]:
#path = '../'
extracted_data = load_pdf_data("../data/yt_test")


In [20]:
#create chunks of text
def text_chunk_splitter(extracted_data):
    text_split = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunk = text_split.split_documents(extracted_data)

    return text_chunk

In [21]:
text_chunks = text_chunk_splitter(extracted_data)
print("chunk length:  ", len(text_chunks))

chunk length:   168


In [24]:
#text_chunks[0]

Document(page_content="1\n00:00:00,120 --> 00:01:42,119\nthis speaker outperformed Tony Robbins last year he was more popular and people love Tony\nRobbins it's no put in not sailing against Tony he's amazing I love the guy but people said he was\nbetter so let me ask you a question imagine you're living in your dream home you got the cabin for\nthe family the beach house your bills are paid you've built something big but you don't have your", metadata={'source': '../data/yt_test/gary_timeSt_yt_api.pdf', 'page': 0})

In [25]:
#download embedding model
def download_embedding_model():
    embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embedding


In [26]:
embedding = download_embedding_model()

# hybrid search exper.

In [ ]:
from pinecone_text.sparse import BM25Encoder
from langchain_community.retrievers import (
    PineconeHybridSearchRetriever,
)

In [ ]:
list_chunks=[x.page_content for x in text_chunks]

In [ ]:
# use default tf-idf values for sparse encoder
bm25_encoder = BM25Encoder().default()

In [ ]:
bm25_encoder.fit(list_chunks)

In [ ]:
#store the values into json file format
bm25_encoder.dump("bm25_values.json")
# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

In [ ]:
pinecone.Pinecone(
   api_key=os.getenv("PINECONE_API_KEY"),  
   environment=os.getenv("PINECONE_ENV"),  
)
index_name = "gary-chatbot"
index = pinecone.Index(index_name, host="https://gary-chatbot-mhwhc2g.svc.gcp-starter.pinecone.io")

Run this when loading the data in to the vector store for the first time

In [ ]:
# Run this line when loading the embedding to the vector store first time
#docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [ ]:
# retriever = PineconeHybridSearchRetriever(
#     embeddings=embedding, sparse_encoder=bm25_encoder, index=index
# )

pinecone hybrid search will give an error

In [ ]:
# query = "what is allergy?"
# #retiriever = hybrid_retriever.as_retriever(search_kwargs=dict(k=3))

# docs = retriever.get_relevant_documents(query)

# #print("Results", docs)

In [ ]:
# query = "what is allergy?"
# #retiriever = hybrid_retriever.as_retriever(search_kwargs=dict(k=3))

# #docs = retriever.get_relevant_documents(query)
# docs = await retriever.aget_relevant_documents(query)
# #print("Results", docs)

In [27]:
query_res = embedding.embed_query("Hi today is March 12 Tuesday")
print("Length: ", len(query_res))

Length:  384


In [ ]:
from langchain_pinecone import PineconeVectorStore

In [28]:
pinecone.Pinecone(
   api_key=os.getenv("PINECONE_API_KEY"),  
   environment=os.getenv("PINECONE_ENV"),  
)
index_name = "gary-chatbotv2"

In [29]:
# Run this line when loading the embedding to the vector store first time
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [30]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embedding)
query = "insulin?"
retiriever = docsearch.as_retriever(search_kwargs=dict(k=3))
docs = retiriever.get_relevant_documents(query)

print("Results", docs)
docs[0].page_content
#[doc.page_content for doc in docs]

Results [Document(page_content="crushed brown sugar and natural honey as a snack yogurt with fruit on the bottom 44 grams of\nsugar right you know what that is that's an insulin dependency diet do you know that Pfizer just paid\n6.6 billion dollars for arena Pharmaceuticals you know what arena Pharmaceuticals does anybody\nknow fixes myocarditis pericarditis and diffuse vasculitis as a consequence of vaccine injury that's a\nfact so we need to understand that sometimes the powers to be don't necessarily have our best"), Document(page_content="and these can be fixed these can be supplemented for and they can be fixed so we do not need to\nwalk around with the kind of ailments like we do in society if you look at the trends in modern\nmedicine you know diabetes is a 110 billion dollar annual industry do you know that 110 billion\ndollars a year do you think that there's a meeting going on somewhere in a big Pharma boardroom\nright now to put that out of business oh no you know if you sai

"crushed brown sugar and natural honey as a snack yogurt with fruit on the bottom 44 grams of\nsugar right you know what that is that's an insulin dependency diet do you know that Pfizer just paid\n6.6 billion dollars for arena Pharmaceuticals you know what arena Pharmaceuticals does anybody\nknow fixes myocarditis pericarditis and diffuse vasculitis as a consequence of vaccine injury that's a\nfact so we need to understand that sometimes the powers to be don't necessarily have our best"

In [31]:
prompt_template = """ 
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.


Context : {context}
Question: {question}

Only return helpful answer below and nothing else.
Helpful answer:
"""

In [32]:
context = "\n\n".join([doc.page_content for doc in docs])
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
#PROMPT = ChatPromptTemplate(prompt_template) #,input_variables=["context", "question"])

#prompt = PROMPT.format(context=context,question= query) 
chain_type_kwargs = {"prompt": PROMPT}

In [37]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [38]:
from llama_cpp import Llama

## N.B
CMAKE_ARGS="-DLLAMA_OPENBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.56


In [39]:
# llm = LlamaCpp(
#     model_path= "../model/gguf/llama-2-7b-chat.Q3_K_M.gguf",
#     n_gpu_layers= -1,
#     n_batch= 512,
#     max_tokens= 512,
#     temperature=0.7,
#     callback_manager=callback_manager,
#     #verbose=True # Verbose is required to pass to the callback manager
# )

llm = LlamaCpp(
    model_path= "../model/gguf/llama-2-7b-chat.Q3_K_M.gguf",
    n_gpu_layers= -1,
    n_batch= 512,
    max_tokens= 512,
    temperature=0.7,
    callback_manager=callback_manager,
    #verbose=True # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../model/gguf/llama-2-7b-chat.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32  

ValidationError: 1 validation error for LlamaCpp
callback_manager
  instance of BaseCallbackManager expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseCallbackManager)

In [ ]:
QA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retiriever,
    #retriever = docsearch.as_retriever(search_kwargs={'k':3}),
    return_source_documents = True,
    chain_type_kwargs=chain_type_kwargs
)


In [ ]:
import textwrap

In [ ]:
import sys
while True:
    user_input = input(f"Input prompt: ")
    if user_input == 'exit':
        print("Exiting")
        sys.exit()
    if user_input == ' ':
        continue
    result = QA({'query': user_input})
    result = textwrap.fill(result["result"], width=50)
    #print(f"Answer: {wraped_result['wraped_result']}")
    print(f"Answer: {result}")

In [ ]:
# while True:
#     user_Input: input(f"Input prompt: ")
#     result = QA({"query": user_Input})
#     print("Response: ", result["result"])